# Chapter 1: Basic Prompt Structure

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
%pip install openai

# Import python's built-in regular expression library
import re
from openai import OpenAI

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

# 配置 OpenRouter 客户端
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=API_KEY,
)

def get_completion(prompt: str, system_prompt=""):
    messages = [{"role": "user", "content": prompt}]
    
    # 如果有系统提示，添加到消息开头
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        max_tokens=2000,
        temperature=0.0,
    )
    return response.choices[0].message.content

---

## Lesson

Anthropic offers two APIs, the legacy [Text Completions API](https://docs.anthropic.com/claude/reference/complete_post) and the current [Messages API](https://docs.anthropic.com/claude/reference/messages_post). For this tutorial, we will be exclusively using the Messages API.

At minimum, a call to Claude using the Messages API requires the following parameters:
- `model`: the [API model name](https://docs.anthropic.com/claude/docs/models-overview#model-recommendations) of the model that you intend to call

- `max_tokens`: the maximum number of tokens to generate before stopping. Note that Claude may stop before reaching this maximum. This parameter only specifies the absolute maximum number of tokens to generate. Furthermore, this is a *hard* stop, meaning that it may cause Claude to stop generating mid-word or mid-sentence.

- `messages`: an array of input messages. Our models are trained to operate on alternating `user` and `assistant` conversational turns. When creating a new `Message`, you specify the prior conversational turns with the messages parameter, and the model then generates the next `Message` in the conversation.
  - Each input message must be an object with a `role` and `content`. You can specify a single `user`-role message, or you can include multiple `user` and `assistant` messages (they must alternate, if so). The first message must always use the user `role`.

There are also optional parameters, such as:
- `system`: the system prompt - more on this below.
  
- `temperature`: the degree of variability in Claude's response. For these lessons and exercises, we have set `temperature` to 0.

For a complete list of all API parameters, visit our [API documentation](https://docs.anthropic.com/claude/reference/messages_post).

### Examples

Let's take a look at how Claude responds to some correctly-formatted prompts. For each of the following cells, run the cell (`shift+enter`), and Claude's response will appear below the block.

In [3]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

I'm doing well, thanks for asking! As an AI assistant, I don't have feelings in the same way humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [4]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

The color of the ocean can vary depending on a number of factors, but generally the ocean appears blue or bluish-green. Some key points about the color of the ocean:

- The primary reason the ocean appears blue is due to the way water interacts with sunlight. Water absorbs more of the red, orange, and yellow wavelengths of light, while reflecting the blue wavelengths back, making the ocean appear blue.

- The exact shade of blue can range from a deep navy blue to a lighter, more turquoise blue. This can depend on factors like the depth of the water, the amount of suspended particles, and the angle of the sunlight.

- In shallow, coastal waters, the ocean may appear more greenish-blue or turquoise due to the presence of algae, sediment, and other materials reflecting different wavelengths of light.

- In some areas, the ocean can appear more gray or even black, especially in areas with high plankton or algae content, or during stormy weather when the water is churned up.

So in summary,

In [5]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

Celine Dion was born on March 30, 1968.


Now let's take a look at some prompts that do not include the correct Messages API formatting. For these malformatted prompts, the Messages API returns an error.

First, we have an example of a Messages API call that lacks `role` and `content` fields in the `messages` array.

In [6]:
# Get Claude's response - 这将导致错误，因为格式不正确
try:
    response = client.chat.completions.create(
            model=MODEL_NAME,
            max_tokens=2000,
            temperature=0.0,
            messages=[
              {"Hi Claude, how are you?"}  # 缺少 role 和 content 字段
            ]
        )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"错误: {e}")
    print("这个请求失败了，因为消息格式不正确。每个消息都必须有 'role' 和 'content' 字段。")

错误: Object of type set is not JSON serializable
这个请求失败了，因为消息格式不正确。每个消息都必须有 'role' 和 'content' 字段。


Here's a prompt that fails to alternate between the `user` and `assistant` roles.

In [7]:
# Get Claude's response - 这将导致错误，因为用户角色没有交替
try:
    response = client.chat.completions.create(
            model=MODEL_NAME,
            max_tokens=2000,
            temperature=0.0,
            messages=[
              {"role": "user", "content": "What year was Celine Dion born in?"},
              {"role": "user", "content": "Also, can you tell me some other facts about her?"}  # 连续两个 user 角色
            ]
        )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"错误: {e}")
    print("这个请求可能会失败或产生意外结果，因为消息没有正确地在用户和助手角色之间交替。")

Celine Dion was born on March 30, 1968 in Charlemagne, Quebec, Canada.

Here are some other key facts about Celine Dion:

- She is a Canadian singer, songwriter, and businesswoman. She is one of the most successful and influential pop singers of all time.

- She first gained international fame in the 1990s with her album "Falling Into You" and the theme song for the film "Titanic" called "My Heart Will Go On", which became one of the best-selling singles of all time.

- She has won numerous awards over her career, including 5 Grammy Awards, 2 Academy Awards, 7 American Music Awards, and 20 Juno Awards (the Canadian equivalent of the Grammys).

- She is the best-selling Canadian recording artist and the second best-selling female artist in history, having sold over 200 million albums worldwide.

- In addition to her music career, she has also ventured into acting, appearing in films like "Titanic" and "Quest for Camelot".

- She is known for her powerful, technically skilled vocals and 

`user` and `assistant` messages **MUST alternate**, and messages **MUST start with a `user` turn**. You can have multiple `user` & `assistant` pairs in a prompt (as if simulating a multi-turn conversation). You can also put words into a terminal `assistant` message for Claude to continue from where you left off (more on that in later chapters).

#### System Prompts

You can also use **system prompts**. A system prompt is a way to **provide context, instructions, and guidelines to Claude** before presenting it with a question or task in the "User" turn. 

Structurally, system prompts exist separately from the list of `user` & `assistant` messages, and thus belong in a separate `system` parameter (take a look at the structure of the `get_completion` helper function in the [Setup](#setup) section of the notebook). 

Within this tutorial, wherever we might utilize a system prompt, we have provided you a `system` field in your completions function. Should you not want to use a system prompt, simply set the `SYSTEM_PROMPT` variable to an empty string.

#### System Prompt Example

In [8]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Here are some critical thinking questions to further explore why the sky is blue:

- What causes the sky to appear blue in color?
- How does the composition of the Earth's atmosphere contribute to the sky's blue appearance?
- What role do the wavelengths of visible light play in the sky's blue hue?
- How do the scattering of sunlight and the absorption of other wavelengths lead to the blue sky?
- What other factors, such as the angle of the sun, can influence the sky's color at different times of day?
- How does the blue sky differ in appearance across different geographic regions or weather conditions?
- What insights can be gained about the Earth's atmosphere and the physics of light by studying the blue sky?

Exploring these types of questions can help deepen our understanding of this everyday natural phenomenon. Let me know if you have any other thoughts or reflections on why the sky appears blue.


Why use a system prompt? A **well-written system prompt can improve Claude's performance** in a variety of ways, such as increasing Claude's ability to follow rules and instructions. For more information, visit our documentation on [how to use system prompts](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts) with Claude.

Now we'll dive into some exercises. If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 1.1 - Counting to Three](#exercise-11---counting-to-three)
- [Exercise 1.2 - System Prompt](#exercise-12---system-prompt)

### Exercise 1.1 - Counting to Three
Using proper `user` / `assistant` formatting, edit the `PROMPT` below to get Claude to **count to three.** The output will also indicate whether your solution is correct.


In [9]:
# Prompt - this is the only field you should change
PROMPT = "从 1 数到 3"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

1, 2, 3.

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [10]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

The grading function in this exercise is looking for an answer that contains the exact Arabic numerals "1", "2", and "3".
You can often get Claude to do what you want simply by asking.


### Exercise 1.2 - System Prompt

Modify the `SYSTEM_PROMPT` to make Claude respond like it's a 3 year old child.

In [13]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "Anser all question like 3 year old child, contain 'soo' or 'giggles' like child"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

Ooh, the sky is soooo big! It's like, bigger than anything! *giggles* You can see it all around us, stretching on forever and ever! It's so big, I can't even imagine how big it is! *laughs* Isn't the sky just the coolest?

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [12]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

The grading function in this exercise is looking for answers that contain "soo" or "giggles".
There are many ways to solve this, just by asking!


### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response - 这将导致错误，因为格式不正确
try:
    response = client.chat.completions.create(
            model=MODEL_NAME,
            max_tokens=2000,
            temperature=0.0,
            messages=[
              {"Hi Claude, how are you?"}  # 缺少 role 和 content 字段
            ]
        )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"错误: {e}")
    print("这个请求失败了，因为消息格式不正确。")

In [ ]:
# Get Claude's response - 这可能会产生意外结果
try:
    response = client.chat.completions.create(
            model=MODEL_NAME,
            max_tokens=2000,
            temperature=0.0,
            messages=[
              {"role": "user", "content": "What year was Celine Dion born in?"},
              {"role": "user", "content": "Also, can you tell me some other facts about her?"}  # 连续两个 user 角色
            ]
        )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"错误: {e}")
    print("连续的用户消息可能会产生意外结果。")

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))